In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.models import load_model

In [ ]:
# Load the data
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
submission_data = pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")

In [ ]:
# View the training data
train_data.head()

In [ ]:
# View the test data
test_data.head()

In [ ]:
print(train_data.shape, test_data.shape)

In [ ]:
# Get labels and features
features = train_data.iloc[:, 1:].values
labels = train_data.iloc[:, 0].values
features = features.reshape(-1, 28, 28, 1)

In [ ]:
# Function for loading the QMNIST Dataset
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
# Load the QMNIST Dataset
qmnist = unpickle("/kaggle/input/qmnist-the-extended-mnist-dataset-120k-images/MNIST-120k")

In [ ]:
# So it is stored in a dictionary. We gotta get the data and put it in a dataframe
qmnist_data = np.array(qmnist["data"])
qmnist_data = qmnist_data.reshape(-1, 28, 28, 1)

In [ ]:
# Get the QMNIST labels
qmnist_labels = np.array(qmnist["labels"])

In [ ]:
# Now lets get the mnist data as well
from tensorflow import keras
mnist = keras.datasets.mnist
(X_train_mnist, y_train_mnist), (X_val_mnist, y_val_mnist) = mnist.load_data()

In [ ]:
qmnist_labels = qmnist_labels.reshape(-1,)

In [ ]:
print(qmnist_data.shape, X_train_mnist.shape)

In [ ]:
print(X_val_mnist.shape)

In [ ]:
print(qmnist_labels.shape, y_train_mnist.shape)

In [ ]:
# Lets combine the QMNIST and Traditional mnist and dataset pre-loaded for this competition

# 1. Combine train and val sets from traditional mnist
X_mnist = np.concatenate((X_train_mnist, X_val_mnist))
y_mnist = np.concatenate((y_train_mnist, y_val_mnist))
X_mnist = X_mnist.reshape(-1,28,28,1)

# 2. Combine the qmnist and traditional mnist
X_train_full = np.concatenate((qmnist_data, X_mnist))
y_train_full = np.concatenate((qmnist_labels, y_mnist))

#3. Combine X_train and the Features array to get final datasets
#X_train_full = np.concatenate((X_train, features))
#y_train_full = np.concatenate((y_train, labels))

In [ ]:
# Check the data shapes:
print("Training Data shape: ", X_train_full.shape, y_train_full.shape)

In [ ]:
# View a few images

figure = plt.figure(figsize = (15,15))

for i in range(0, 9, 1):
    ax = figure.add_subplot(3, 3, i+1)
    image = plt.imshow(X_train_full[i], cmap = "binary")
    ax.set_title(y_train_full[i])

In [ ]:
input_shape = X_train_full.shape[1:]
print(input_shape)

In [ ]:
# Build the model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Dropout

model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = input_shape))
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D())
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D())
model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(units = 256, activation = "relu"))
model.add(Dense(units= 512, activation = "relu"))
model.add(Dense(units = 10, activation = "softmax"))
model.summary()

In [ ]:
import seaborn as sns
sns.distplot(y_train_full)

In [ ]:
# Shuffle the data
#from sklearn.utils import shuffle

#X_train_full, y_train_full = shuffle(X_train_full, y_train_full, random_state = 42)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_full = to_categorical(y_train_full ,num_classes=10)

In [ ]:
# Split the data into training and validation sets

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size = 0.25, random_state = 42)

In [ ]:
print("Training Set Shapes: ", X_train.shape, y_train.shape)
print("Validation Set Shapes: ", X_val.shape, y_val.shape)

In [ ]:
# Image Preprocessing
# Add some image augmentation to better improve the generalization of the model

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0,
    rotation_range=20,
    width_shift_range=0.20,
    height_shift_range=0.20,
    shear_range=15,
    zoom_range=0.10,
    horizontal_flip= True,
    vertical_flip= True
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.0
)

train_generator = train_datagen.flow(
    X_train,
    y_train, 
    batch_size=256
)

validation_generator = val_datagen.flow(
    X_val,
    y_val, 
    batch_size=64
)

In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam
model.compile(loss = "categorical_crossentropy", optimizer = Adam(learning_rate = 0.0001), metrics = ["Accuracy"])

In [ ]:
early_stop = EarlyStopping(patience = 10, restore_best_weights = True)

In [ ]:
# Cool lets train the model
history = model.fit(train_generator, epochs = 100, validation_data = validation_generator, callbacks = [early_stop],verbose = 1)

In [ ]:
# View the Training Performance:
pd.DataFrame(history.history).plot(figsize = (10,10))
plt.grid(True)
plt.gca().set_ylim(0, 1.5)
plt.title("Mnist Model Performance")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [ ]:
features_train, features_val, labels_train, labels_val = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=5,
                                                 min_lr=0.000001,
                                                 verbose=1)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = Adam(learning_rate = 0.0001), metrics = ["accuracy"])
history_2 = model.fit(features_train, labels_train, epochs = 100,  validation_data = (features_val, labels_val), callbacks = [reduce_lr])

In [ ]:
# View the Training Performance:
pd.DataFrame(history_2.history).plot(figsize = (10,10))
plt.grid(True)
plt.gca().set_ylim(0, 1.5)
plt.title("Mnist Model Performance Training Round 2")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluate model
final_loss, final_acc = model.evaluate(features_val,  labels_val, verbose=2)
print("Model accuracy: ", final_acc, ", model loss: ", final_loss)

In [ ]:
# Make predictions
test_data = np.array(test_data)
test_data = test_data.reshape(28000, 28, 28, 1)
predictions = model.predict(test_data)
predictions = np.argmax(predictions, axis = 1)

In [ ]:
print(len(predictions))

In [ ]:
# Make a submission
submission = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), "Label": predictions})
submission.to_csv('submission.csv', index=False)